# Signal Synchronization


In [ ]:
import io

import matplotlib.pyplot as plt

import pluto_sdr_pr.ioutils
import pluto_sdr_pr.processing
import pluto_sdr_pr.signal

import numpy as np

## Input preparation

First, prepare the input sample streams. Print some meta-data about the selected recordings.


In [ ]:
ref_file_path = "data/pluto_a_ref.2021-08-13T17_17_19_487.sdriq"
surv_file_path = "data/pluto_b_surv.2021-08-13T17_17_22_603.sdriq"

ref = pluto_sdr_pr.ioutils.SdriqSampleIO(ref_file_path)
surv = pluto_sdr_pr.ioutils.SdriqSampleIO(surv_file_path)
enb = pluto_sdr_pr.signal.ENodeB(6)
mss = pluto_sdr_pr.signal.MultiSignalStream()

ref_end, surv_end = ref.seek(0, io.SEEK_END), surv.seek(0, io.SEEK_END)

print(
    f"There are {ref_end} samples in reference and {surv_end} in surveillance "
    "channel."
)
num_ref_frames = int(
    ref_end // (ref.sample_rate * pluto_sdr_pr.signal.ENodeB.T_FRAME)
)

num_surv_frames = int(
    surv_end // (surv.sample_rate * pluto_sdr_pr.signal.ENodeB.T_FRAME)
)

print(
    f"This equates to roughly {ref_end / ref.sample_rate:.3f} & "
    f"{surv_end / surv.sample_rate:.3f} sec of data or "
    f"{num_ref_frames} & {num_surv_frames} frames"
)

## PSS-Only Synchronization

First synchronize only based on PSS-correlation. The resulting cell id will tell us only the PSS component of the actual cell id.

The result will be a roughly synchronized sample stream.


In [ ]:
ref.seek(0), surv.seek(0)
num_frames = 12
cell_id, pss_correlations, _ = mss.start_synchronized(
    ref,
    surv,
    enb=enb,
    num_frames=num_frames,
    pss_only=True,
)
print(f"Cell ID: {cell_id}")

samples = mss.read(num_samples=10)
print(f"Read {samples[0].shape[0]} samples from inputs "
      f"{[str(sample_block.source) for sample_block in samples]}"
)

## Plot Correlation Results

The synchronization function provides the correlation results as debug output. We can plot those to visually verify the result.


In [ ]:
num_pss = len(pss_correlations)
fig, axs = plt.subplots(
    num_pss, pss_correlations[0].magnitudes.shape[0], figsize=(20, 10)
)

for ax_row, correlations in zip(axs, pss_correlations):
    for idx, (ax, corr) in enumerate(zip(ax_row, correlations.magnitudes)):
        norm_corr = (
            corr / correlations.max_magnitude[correlations.max_peak_index[1]]
        )
        time = np.arange(norm_corr.shape[0]) / correlations.sample_rate
        ax.plot(time, norm_corr, "tab:red" if idx == 1 else "tab:blue")
        ax.set_xticks(
            np.linspace(0, num_frames * pluto_sdr_pr.signal.ENodeB.T_FRAME, 6)
        )
        ax.set_yticks(np.linspace(0, 1, 6))
        ax.set_ylim(0, 1.1)
        ax.grid(True)